### Importing dependencies


In [ ]:
# Dependencies

import os
import pickle

import requests
import json

from io import BytesIO
# plt
import matplotlib.pyplot as plt

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from PIL import Image
from IPython.display import display

# Constants
SCOPES = ['https://www.googleapis.com/auth/photoslibrary.readonly']
API_ENDPOINT = 'https://photoslibrary.googleapis.com'
OAUTH_CALLBACK = 'http://localhost:8080/'
CLIENT_SECRET_FILE = 'oauth_client_secret.json'
APPLICATION_NAME = 'Photos Playground'

In [ ]:
def get_credentials():
    # Authenticate the user
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=8080)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds


In [ ]:
# Get credentials for Google Photos by opening the browser and asking the user to authenticate
creds=get_credentials()

# We will not need to authenticate again because we will save the credentials in a file

### Listing the latest pictures/videos in the library and displaying 1


In [ ]:
r=requests.get(API_ENDPOINT+'/v1/mediaItems',headers={'Authorization':'Bearer '+creds.token})
content=r.json()
mediaItems=content['mediaItems']
print('Number of media items: '+str(len(mediaItems)))
print('First media item: ')
print(json.dumps(mediaItems[0], indent=2))

img_data = requests.get(mediaItems[0]['baseUrl']+'=w640-h480').content
img = Image.open(BytesIO(img_data))
plt.axis('off')
plt.imshow(img)

### Listing photos with PEOPLE content category


In [ ]:
r=requests.post(
    API_ENDPOINT+'/v1/mediaItems:search',
    headers={'Authorization':'Bearer '+creds.token},
    json={
          'filters':{
            "contentFilter":
              {"includedContentCategories":["PEOPLE"]},
            "mediaTypeFilter":
              {"mediaTypes":["PHOTO"]}
          }
    } 
  )
content=r.json()
mediaItems=content['mediaItems']
print('Number of media items: '+str(len(mediaItems)))
print('First media item: ')
print(json.dumps(mediaItems[0], indent=2))



img_data = requests.get(mediaItems[0]['baseUrl']+'=w640-h480').content
img = Image.open(BytesIO(img_data))
plt.axis('off')
plt.imshow(img)


### Getting the entirety of the favorite photos


In [ ]:
# We are going to have to loop through all the images in the favorites
page_token = ''
contents=[]
while True:
    r=requests.post(
        API_ENDPOINT+'/v1/mediaItems:search',
        headers={'Authorization':'Bearer '+creds.token},
        json={'filters':
                {
                    "featureFilter": {
                        "includedFeatures": [
                            "FAVORITES"
                        ]
                    },
                    "mediaTypeFilter": {
                        "mediaTypes":["PHOTO"]
                    },

                },
                'pageSize':100,
                'pageToken': page_token
        }
    )
    content=r.json()
    contents.append(content)
    if 'nextPageToken' not in content:
        break
    page_token=content['nextPageToken']

mediaItems=[]
for content in contents:
    mediaItems+=content['mediaItems']

print('Number of media items: '+str(len(mediaItems)))

# Display the first 10 items
fig = plt.figure(figsize=(16, 16))
for i in range(10):
    img_data = requests.get(mediaItems[i]['baseUrl']+'=w640-h480').content
    img = Image.open(BytesIO(img_data))
    fig.add_subplot(5, 2, i+1)
    plt.axis('off')
    plt.imshow(img)

plt.show()
